In [23]:
import pandas as pd
import pyperclip
import sacrebleu
from pathlib import Path

"""https://arxiv.org/pdf/2404.13813"""

In [2]:
def load_data(dataset_path):
    parent_path = Path(dataset_path).parent

    if not (parent_path / 'train_split.csv').exists():
        df = pd.read_csv(dataset_path)

        df = df.rename(columns={ # TODO: This is bulshit, remove it
            'target': 'mansi',
            'source': 'ru'
        })

        shuffled_df = df.sample(frac=1).reset_index(drop=True)

        total_size = len(shuffled_df)
        test_size = int(total_size * 0.1)
        val_size = int(total_size * 0.1)
        # train_size = total_size - test_size - val_size

        test_df = shuffled_df[:test_size]
        val_df = shuffled_df[test_size: test_size + val_size]
        train_df = shuffled_df[test_size + val_size:]

        train_df.to_csv(parent_path / 'train_split.csv', index=False)
        val_df.to_csv(parent_path / 'val_split.csv', index=False)
        test_df.to_csv(parent_path / 'test_split.csv', index=False)
    else:
        train_df = pd.read_csv(parent_path / 'train_split.csv')
        val_df = pd.read_csv(parent_path / 'val_split.csv')
        test_df = pd.read_csv(parent_path / 'test_split.csv')

    print(f"Train size: {len(train_df)}")
    print(f"Val size: {len(val_df)}")
    print(f"Test size: {len(test_df)}")

    return train_df, val_df, test_df


train_df, val_df, test_df = load_data("data/cleared_v2.csv")

Train size: 63947
Val size: 7993
Test size: 7993


In [30]:
n_train_pairs = 10000
start_pred_pairs_index = 0
end_pred_pairs_index = 50

train_df_pairs = train_df.sample(n_train_pairs)


str_pairs = ""
for index, row in train_df_pairs.iterrows():
    # str_pairs += f"Russian: {row['ru']}\nMansi: {row['mansi']}\n\n"
    str_pairs += f"Mansi: {row['mansi']} Russian: {row['ru']}\n"


prompt = \
"""
You are a translator. Here is pairs of Mansi sentences and their translations in Russian language:
{data}

Your task is to translate this {n_sentences} sentences from Mansi to Russian:
{sentences}

Return the translated sentence in Russian language as Python list.
"""
# sample_value = test_df.sample(1)['ru'].iloc[0]
# sample_values = test_df.sample(n_pred_pairs).reset_index()
sample_values = test_df[start_pred_pairs_index:end_pred_pairs_index]

str_ru = ""
tgt_texts = []

for index, row in sample_values.iterrows():
    str_ru += f"Mansi: {row['mansi']}\n"
    tgt_texts.append(row['ru'])

sample_value = str_ru


filled_prompt = prompt.format(data=str_pairs, n_sentences=len(sample_values), sentences=sample_value)
# print(filled_prompt)

pyperclip.copy(filled_prompt)



In [32]:
mansi_translations = [
    'Красивый день.',
    'Из костей его крылья сделаны.',
    'Сколько женщин там было.',
    'Девочка любила маму и папу.',
    'Только Настя всё время рядом с ним сидит, из-за этого Силька её ревнует.',
    'Я молча сижу.',
    'БЕЛОЕ И БЕЛОЕ ДИТЯ.',
    'Золотой царь подать свою собирает.',
    'Много лет спустя, племянница крышку подняла.',
    'Потом тихо стал.',
    'У отца однажды случилась болезнь, умер.',
    'Поднялся, посмотрел на небо.',
    'Почти восемьдесят процентов населения России себя русскими считают.',
    'Лягушка людям говорит: «К Менкву-великану отвезите, отдайте его».',
    'Семья в 1958 году в Саранпауль переехала.',
    'Его шею тоже перерубил.',
    'Высокий мужчина с отцом жил.',
    'Начинать охоту.',
    'По программе выделенных денег в Нефтеюганском районе на базе отдыха «Сказка» местные жители на калданках соревновались.',
    'За сделанную им работу женщина очень радовалась.',
    'На этом моем пальце, с прочными деревьями, лесная грива пусть будет!',
    'Вначале пусть хорошо подумают, потом пусть обратно скажут.',
    'Про маму и про папу я рассказ напишу.',
    'Жить научился.',
    'Поэтому он самый главный, много о нём сплетен.',
    'Наверх, оказывается, вода, вниз, оказывается, вода, меня, зверя, вверх несет, меня зверя, вниз несет.',
    'Я, Похсын Хар Товыль Мольсянгынг Отыр, и ты возьми! (название легенды)',
    'Евдокия Андреевна в учебное время «Русско-хантыйский словарь» выпустила.',
    'Борис Хохряков, председатель окружной Думы, первое слово о сделанной работе за 2015 год сказал.',
    'Ещё в дом.',
    'Под святой землей с грязной спиной Вниз опустись!',
    'Кроме этого дети про охрану природы стихи писали.',
    'Когда утки домой возвращаются, я им зерна насыплю.',
    'И ты у отца моего золотая дочь.',
    'Какой старый очаг в лесу найду, на этом очаге огонь разжигать можно?',
    'Поселковая улица совсем большой стала - отец каждый раз в селе сено косит.',
    'Мыслит как-то.',
    'До сегодняшнего дня я это помню.',
    'Машенька засмеялась - никто не отзывается.',
    'Вошёл в кабинет к начальнику.',
    'После этого в 2013 году Петр Валерьевич в городе Ханты-Мансийске по контракту в армию служить начал в "Радиотехнической связи".',
    'Эту болезнь вылечить не могут.',
    'Вышли на берег.',
    'Когда домой вернулись, мама мне сказала, что там кто-то просится войти, голову чешет, потом как уснул.',
    'Такой Пик был, по деревьям лазил, всё смотрел.',
    'Мы с ним жили, я к своим родственникам, жену искать поеду.',
    'Другие люди напугали, манси так два брата отпускают.',
    'Гей!.. Гей!.. По степи шагает Красная Армия.',
    'Первые ребята, приехавшие на форум, были из деревни Ванзеват Белоярского района.',
    'С чёрного пластика плёнка очень тонкая, но она хорошо отражает солнечные лучи, поэтому плёнку чёрного цвета используют, чтобы в теплицах было теплее, и чтобы созревание урожая ускорить.',
]
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++

bleu_score = bleu_calc.corpus_score(mansi_translations, [tgt_texts])
chrf_score = chrf_calc.corpus_score(mansi_translations, [tgt_texts])

print(bleu_score)
print(chrf_score)

BLEU = 13.74 41.4/15.5/9.1/6.1 (BP = 1.000 ratio = 1.060 hyp_len = 495 ref_len = 467)
chrF2++ = 36.88
